In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%bash
pip3 show kaggle_runner || ( git clone https://github.com/pennz/kaggle_runner; \
mv kaggle_runner k && \
mv k/* . && mv k/.* ; \
pip3 install -e .;\
export PATH=$PWD/bin:$PATH; \
entry.sh &)

Name: kaggle-runner
Version: 0.1.6
Summary: Run kaggle kernels, for fast model prototyping.
Home-page: http://github.com/pennz/kaggle_runner
Author: pennz
Author-email: pengyuzhou.work@gmail.com
License: MIT
Location: /content
Requires: slug, parse, python-logging-rabbitmq, kaggle
Required-by: 


In [3]:
!make xla
import torch_xla
from importlib import reload
import kaggle_runner
reload(kaggle_runner)

make: command: Command not found
make: type: Command not found
make: type: Command not found
python3 -m pip show torch_xla || ( curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py; \
python3 pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev; \
python3 -m pip install *.whl; \
python3 -m pip install transformers==2.5.1 > /dev/null; \
python3 -m pip install pandarallel > /dev/null; \
python3 -m pip install catalyst==20.4.2 > /dev/null;)
Name: torch-xla
Version: 1.6+2b2085a
Summary: XLA bridge for PyTorch
Home-page: https://github.com/pytorch/xla
Author: PyTorch/XLA Dev Team
Author-email: pytorch-xla@googlegroups.com
License: UNKNOWN
Location: /usr/local/lib/python3.6/dist-packages
Requires: 
Required-by: 


<module 'kaggle_runner' from '/content/kaggle_runner/__init__.py'>

In [4]:
import numpy as np
import pandas as pd
import os
os.environ['XLA_USE_BF16'] = "1"

In [5]:
from glob import glob

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.autograd import Variable
from torch.utils.data.sampler import SequentialSampler, RandomSampler
import sklearn

In [7]:
import time
import random
from datetime import datetime
from tqdm import tqdm
tqdm.pandas()

In [8]:
from transformers import XLMRobertaModel, XLMRobertaTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup, get_constant_schedule
from fastai.text.transform import Vocab
from catalyst.data.sampler import DistributedSamplerWrapper, BalanceClassSampler

In [9]:
import gc
import re

In [10]:
# !python3 -m pip install nltk > /dev/null
import nltk
nltk.download('punkt')

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
from nltk import sent_tokenize

In [12]:
from pandarallel import pandarallel

In [13]:
pandarallel.initialize(nb_workers=4, progress_bar=False)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [14]:
from fastai.basic_data import DataBunch
from kaggle_runner.kernels.fastai_kernel import FastAIKernel
from kaggle_runner import may_debug

[('__call__', <function LevelMapper.__call__ at 0x7f0d9aeb6620>), ('__init__', <function LevelMapper.__init__ at 0x7f0d9aeb6598>)]
[('__call__', <function BalancedPositiveNegativeSampler.__call__ at 0x7f0d9ae002f0>), ('__init__', <function BalancedPositiveNegativeSampler.__init__ at 0x7f0d9ae00268>)]
[('__init__', <function BoxCoder.__init__ at 0x7f0d9ae09730>), ('decode', <function BoxCoder.decode at 0x7f0d9ae098c8>), ('decode_single', <function BoxCoder.decode_single at 0x7f0d9ae09950>), ('encode', <function BoxCoder.encode at 0x7f0d9ae097b8>), ('encode_single', <function BoxCoder.encode_single at 0x7f0d9ae09840>)]
[('__call__', <function Matcher.__call__ at 0x7f0d9ae09620>), ('__init__', <function Matcher.__init__ at 0x7f0d9ae09a60>), ('set_low_quality_matches_', <function Matcher.set_low_quality_matches_ at 0x7f0d9ae096a8>)]
[('__init__', <function ImageList.__init__ at 0x7f0d9ae09bf8>), ('to', <function ImageList.to at 0x7f0d9ae09b70>)]
[('__init__', <function Timebase.__init__ at

In [15]:
SEED = 142

In [16]:
MAX_LENGTH = 224
BACKBONE_PATH = 'xlm-roberta-large'

In [17]:
tokenizer = XLMRobertaTokenizer.from_pretrained(BACKBONE_PATH)

In [18]:
ROOT_PATH = f'/kaggle' # for colab

In [19]:
from kaggle_runner.utils.kernel_utils import get_obj_or_dump
def get_pickled_data(file_path):
    obj = get_obj_or_dump(file_path)

    if obj is None:
        #may_debug(True)

        return get_obj_or_dump(f"{ROOT_PATH}/input/clean-pickle-for-jigsaw-toxicity/{file_path}")

    return obj
vocab = get_pickled_data("vocab.pkl")

if vocab is None: # vocab file read~~
   vocab = [tokenizer.convert_ids_to_tokens(i) for i in range(tokenizer.vocab_size)]
   get_obj_or_dump("vocab.pkl", default=vocab)

In [20]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [21]:
from nltk import sent_tokenize
from random import shuffle
import random
import albumentations
from albumentations.core.transforms_interface import DualTransform, BasicTransform

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject



In [22]:
LANGS = {
    'en': 'english',
    'it': 'italian',
    'fr': 'french',
    'es': 'spanish',
    'tr': 'turkish',
    'ru': 'russian',
    'pt': 'portuguese'
}

In [23]:
def get_sentences(text, lang='en'):
    return sent_tokenize(text, LANGS.get(lang, 'english'))

In [24]:
def exclude_duplicate_sentences(text, lang='en'):
    sentences = []

    for sentence in get_sentences(text, lang):
        sentence = sentence.strip()

        if sentence not in sentences:
            sentences.append(sentence)

    return ' '.join(sentences)

In [25]:
def clean_text(text, lang='en'):
    text = str(text)
    text = re.sub(r'[0-9"]', '', text)
    text = re.sub(r'#[\S]+\b', '', text)
    text = re.sub(r'@[\S]+\b', '', text)
    text = re.sub(r'https?\S+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = exclude_duplicate_sentences(text, lang)

    return text.strip()

In [26]:
class NLPTransform(BasicTransform):
    """ Transform for nlp task."""

    @property
    def targets(self):
        return {"data": self.apply}

    def update_params(self, params, **kwargs):
        if hasattr(self, "interpolation"):
            params["interpolation"] = self.interpolation

        if hasattr(self, "fill_value"):
            params["fill_value"] = self.fill_value

        return params

    def get_sentences(self, text, lang='en'):
        return sent_tokenize(text, LANGS.get(lang, 'english'))

In [27]:
class ShuffleSentencesTransform(NLPTransform):
    """ Do shuffle by sentence """
    def __init__(self, always_apply=False, p=0.5):
        super(ShuffleSentencesTransform, self).__init__(always_apply, p)

    def apply(self, data, **params):
        text, lang = data
        sentences = self.get_sentences(text, lang)
        random.shuffle(sentences)

        return ' '.join(sentences), lang

In [28]:
class ExcludeDuplicateSentencesTransform(NLPTransform):
    """ Exclude equal sentences """
    def __init__(self, always_apply=False, p=0.5):
        super(ExcludeDuplicateSentencesTransform, self).__init__(always_apply, p)

    def apply(self, data, **params):
        text, lang = data
        sentences = []

        for sentence in self.get_sentences(text, lang):
            sentence = sentence.strip()

            if sentence not in sentences:
                sentences.append(sentence)

        return ' '.join(sentences), lang

In [29]:
class ExcludeNumbersTransform(NLPTransform):
    """ exclude any numbers """
    def __init__(self, always_apply=False, p=0.5):
        super(ExcludeNumbersTransform, self).__init__(always_apply, p)

    def apply(self, data, **params):
        text, lang = data
        text = re.sub(r'[0-9]', '', text)
        text = re.sub(r'\s+', ' ', text)

        return text, lang

In [30]:
class ExcludeHashtagsTransform(NLPTransform):
    """ Exclude any hashtags with # """
    def __init__(self, always_apply=False, p=0.5):
        super(ExcludeHashtagsTransform, self).__init__(always_apply, p)

    def apply(self, data, **params):
        text, lang = data
        text = re.sub(r'#[\S]+\b', '', text)
        text = re.sub(r'\s+', ' ', text)

        return text, lang

In [31]:
class ExcludeUsersMentionedTransform(NLPTransform):
    """ Exclude @users """
    def __init__(self, always_apply=False, p=0.5):
        super(ExcludeUsersMentionedTransform, self).__init__(always_apply, p)

    def apply(self, data, **params):
        text, lang = data
        text = re.sub(r'@[\S]+\b', '', text)
        text = re.sub(r'\s+', ' ', text)

        return text, lang

In [32]:
class ExcludeUrlsTransform(NLPTransform):
    """ Exclude urls """
    def __init__(self, always_apply=False, p=0.5):
        super(ExcludeUrlsTransform, self).__init__(always_apply, p)

    def apply(self, data, **params):
        text, lang = data
        text = re.sub(r'https?\S+', '', text)
        text = re.sub(r'\s+', ' ', text)

        return text, lang

In [33]:
def get_open_subtitles():
    df_ot = get_pickled_data("ot.pkl")

    if df_ot is None:
        df_ot = pd.read_csv(f'{ROOT_PATH}/input/open-subtitles-toxic-pseudo-labeling/open-subtitles-synthesic.csv', index_col='id')[['comment_text', 'toxic', 'lang']]
        df_ot = df_ot[~df_ot['comment_text'].isna()]
        df_ot['comment_text'] = df_ot.parallel_apply(lambda x: clean_text(x['comment_text'], x['lang']), axis=1)
        df_ot = df_ot.drop_duplicates(subset='comment_text')
        df_ot['toxic'] = df_ot['toxic'].round().astype(np.int)
        get_obj_or_dump("ot.pkl", default=df_ot)

    return df_ot

In [34]:
class SynthesicOpenSubtitlesTransform(NLPTransform):
    def __init__(self, always_apply=False, supliment_toxic=None, p=0.5, mix=False):
        super(SynthesicOpenSubtitlesTransform, self).__init__(always_apply, p)

        df = get_open_subtitles()
        self.synthesic_toxic = df[df['toxic'] == 1].comment_text.values
        self.synthesic_non_toxic = df[df['toxic'] == 0].comment_text.values

        if supliment_toxic is not None:
            self.synthesic_toxic = np.concatenate((self.synthesic_toxic, supliment_toxic))
        self.mix = mix

        del df
        gc.collect();


    def _mix_both(self, texts):
        for i in range(random.randint(0,2)):
            texts.append(random.choice(self.synthesic_non_toxic))

        for i in range(random.randint(1,3)):
            texts.append(random.choice(self.synthesic_toxic))

    def generate_synthesic_sample(self, text, toxic):
        texts = [text]

        if toxic == 0:
            if self.mix:
                self._mix_both(texts)
                toxic = 1
            else:
                for i in range(random.randint(1,5)):
                    texts.append(random.choice(self.synthesic_non_toxic))
        else:
            self._mix_both(texts)
        random.shuffle(texts)

        return ' '.join(texts), toxic

    def apply(self, data, **params):
        text, toxic = data
        text, toxic = self.generate_synthesic_sample(text, toxic)

        return text, toxic

In [35]:
def get_train_transforms():
    return albumentations.Compose([
        ExcludeUsersMentionedTransform(p=0.95),
        ExcludeUrlsTransform(p=0.95),
        ExcludeNumbersTransform(p=0.95),
        ExcludeHashtagsTransform(p=0.95),
        ExcludeDuplicateSentencesTransform(p=0.95),
    ], p=1.0)

In [36]:
def get_synthesic_transforms(supliment_toxic, p=0.5, mix=False):
    return SynthesicOpenSubtitlesTransform(p=p, supliment_toxic=supliment_toxic, mix=mix)

In [37]:
def get_toxic_comments(df):
        df = df[~df['comment_text'].isna()]
        df = df.drop_duplicates(subset='comment_text')
        df['toxic'] = df['toxic'].round().astype(np.int)

        return df[df['toxic'] == 1].comment_text.values

In [38]:
def onehot(size, target, aux=None):
    if aux is not None:
        vec = np.zeros(size+len(aux), dtype=np.float32)
        vec[target] = 1.
        vec[2:] = aux
        vec = torch.tensor(vec, dtype=torch.float32)
    else:
        vec = torch.zeros(size, dtype=torch.float32)
        vec[target] = 1.

    return vec

In [39]:
class DatasetRetriever(Dataset):
    def __init__(self, labels_or_ids, comment_texts, langs,
                 severe_toxic=None, obscene=None, threat=None, insult=None, identity_hate=None,
                 use_train_transforms=False, test=False, use_aux=True, transformers=None):
        self.test = test
        self.labels_or_ids = labels_or_ids
        self.comment_texts = comment_texts
        self.langs = langs
        self.severe_toxic = severe_toxic
        self.obscene = obscene
        self.threat = threat
        self.insult = insult
        self.identity_hate = identity_hate
        self.use_train_transforms = use_train_transforms
        self.aux = None
        assert transformers is not None
        self.transformers = transformers
        self.vocab = vocab

        if use_aux:
            self.aux = [self.severe_toxic, self.obscene, self.threat, self.insult, self.identity_hate]

    def get_tokens(self, text):
        encoded = self.transformers['tokenizer'].encode_plus(
            text,
            add_special_tokens=True,
            max_length=MAX_LENGTH,
            pad_to_max_length=True
        )

        return encoded['input_ids'], encoded['attention_mask']

    def __len__(self):
        return self.comment_texts.shape[0]

    def __getitem__(self, idx):
        text = self.comment_texts[idx]
        lang = self.langs[idx]

        if self.severe_toxic is None:
            aux = [0., 0., 0., 0., 0.]
        else:
            aux = [self.severe_toxic[idx], self.obscene[idx], self.threat[idx], self.insult[idx], self.identity_hate[idx]]


        label = self.labels_or_ids[idx]

        if self.use_train_transforms and (not self.test):
            text, _ = self.transformers['train_transforms'](data=(text, lang))['data']
            tokens, attention_mask = self.get_tokens(str(text))
            token_length = sum(attention_mask)

            if token_length > 0.8*MAX_LENGTH:
                text, _ = self.transformers['shuffle_transforms'](data=(text, lang))['data']
            elif token_length < 60:
                text, label = self.transformers['synthesic_transforms_often'](data=(text, label))['data']
            else: # will not need to use transforms
                #text, label = synthesic_transforms_low(data=(text, label))['data']
                pass

        # TODO add language detection and shuffle
        # https://pypi.org/project/langdetect/
        # if self.use_train_transforms and self.test:
        #    text, _ = train_transforms(data=(text, lang))['data']
        #    tokens, attention_mask = self.get_tokens(str(text))
        #    token_length = sum(attention_mask)

        #    if token_length > 0.8*MAX_LENGTH:
        #        text, _ = shuffle_transforms(data=(text, lang))['data']
        # to tensors
        tokens, attention_mask = self.get_tokens(str(text))
        tokens, attention_mask = torch.tensor(tokens), torch.tensor(attention_mask)

        if self.test:  # for test, return id TODO TTA
            return [tokens, attention_mask], self.labels_or_ids[idx]

        # label might be changed
        target = onehot(2, label, aux=aux)

        return [tokens, attention_mask], target

    def get_labels(self):
        return list(np.char.add(self.labels_or_ids.astype(str), self.langs))

In [40]:
from kaggle_runner.kernels.fastai_kernel import FastAIKernel

In [41]:
class Shonenkov(FastAIKernel):
    def __init__(self, **kargs):
        super(Shonenkov, self).__init__(**kargs)
        self.data = None
        self.transformers = None
        self.setup_transformers()

    def build_and_set_model(self):
        self.model = ToxicSimpleNNModel()

    def set_random_seed(self):
        seed_everything(SEED)

    def setup_transformers(self):
        if self.transformers is None:
            supliment_toxic = None # avoid overfit
            train_transforms = get_train_transforms();
            synthesic_transforms_often = get_synthesic_transforms(supliment_toxic, p=0.5)
            synthesic_transforms_low = None
            #tokenizer = tokenizer
            shuffle_transforms = ShuffleSentencesTransform(always_apply=True)

            self.transformers = {'train_transforms': train_transforms,
                                 'synthesic_transforms_often': synthesic_transforms_often,
                                 'synthesic_transforms_low': synthesic_transforms_low,
                                 'tokenizer': tokenizer, 'shuffle_transforms':
                                 shuffle_transforms}

    def prepare_train_dev_data(self):
        df_train = get_pickled_data("train.pkl")

        if df_train is None:
            df_train = pd.read_csv(f'{ROOT_PATH}/input/jigsaw-toxicity-train-data-with-aux/train_data.csv')
            df_train['comment_text'] = df_train.parallel_apply(lambda x: clean_text(x['comment_text'], x['lang']), axis=1)
            get_obj_or_dump("train.pkl", default=df_train)

        #supliment_toxic = get_toxic_comments(df_train)
        self.train_dataset = DatasetRetriever(
            labels_or_ids=df_train['toxic'].values,
            comment_texts=df_train['comment_text'].values,
            langs=df_train['lang'].values,
            severe_toxic=df_train['severe_toxic'].values,
            obscene=df_train['obscene'].values,
            threat=df_train['threat'].values,
            insult=df_train['insult'].values,
            identity_hate=df_train['identity_hate'].values,
            use_train_transforms=True,
            transformers=self.transformers
        )
        df_val = get_pickled_data("val.pkl")

        if df_val is None:
            df_val = pd.read_csv(f'{ROOT_PATH}/input/jigsaw-multilingual-toxic-comment-classification/validation.csv', index_col='id')
            df_val['comment_text'] = df_val.parallel_apply(lambda x: clean_text(x['comment_text'], x['lang']), axis=1)
            get_obj_or_dump("val.pkl", default=df_val)

        self.validation_tune_dataset = DatasetRetriever(
            labels_or_ids=df_val['toxic'].values,
            comment_texts=df_val['comment_text'].values,
            langs=df_val['lang'].values,
            use_train_transforms=True,
            transformers=self.transformers
        )
        self.validation_dataset = DatasetRetriever(
            labels_or_ids=df_val['toxic'].values,
            comment_texts=df_val['comment_text'].values,
            langs=df_val['lang'].values,
            use_train_transforms=False,
            transformers=self.transformers
        )

        del df_val
#del df_val_unclean
        gc.collect();

        del df_train
        gc.collect();

    def prepare_test_data(self):
        df_test = get_pickled_data("test.pkl")

        if df_test is None:
            df_test = pd.read_csv(f'{ROOT_PATH}/input/jigsaw-multilingual-toxic-comment-classification/test.csv', index_col='id')
            df_test['comment_text'] = df_test.parallel_apply(lambda x: clean_text(x['content'], x['lang']), axis=1)
            get_obj_or_dump("test.pkl", default=df_test)

        self.test_dataset = DatasetRetriever(
            labels_or_ids=df_test.index.values, ## here different!!!
            comment_texts=df_test['comment_text'].values,
            langs=df_test['lang'].values,
            use_train_transforms=False,
            test=True,
            transformers=self.transformers
        )

        del df_test
        gc.collect();
    def after_prepare_data_hook(self):
        """Put to databunch here"""
        self.data = DataBunch.create(self.train_dataset,
                                     self.validation_dataset,
                                     bs=TrainGlobalConfig.batch_size,
                                     num_workers=TrainGlobalConfig.num_workers)

    def peek_data(self):
        if self.data is not None:
            may_debug()
            o = self.data.one_batch()
            print(o)

            return o
        else:
            if self.logger is not None:
                self.logger.error("peek_data failed, DataBunch is None.")

In [42]:
from kaggle_runner.metrics.metrics import matthews_correlation
class RocAucMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.y_true = np.array([])
        self.y_true_float = np.array([], dtype=np.float)
        self.y_pred = np.array([])
        self.score = 0
        self.mc_score = 0
        self.aux_part = 0

    def update(self, y_true, y_pred, aux_part=0):
        #y_true_ = y_true
        y_true = y_true[:,:2].cpu().numpy().argmax(axis=1)
        y_true_float = y_true.astype(np.float)
        y_pred = nn.functional.softmax(y_pred[:,:2], dim=1).data.cpu().numpy()[:,1]
        self.y_true = np.hstack((self.y_true, y_true))
        self.y_true_float = np.hstack((self.y_true_float, y_true_float))
        self.y_pred = np.hstack((self.y_pred, y_pred))
        try:
            self.score = sklearn.metrics.roc_auc_score(self.y_true, self.y_pred, labels=np.array([0, 1]))
        except Exception:
            self.score = 0
        self.mc_score = matthews_correlation(self.y_true_float, self.y_pred)
        self.aux_part = aux_part

    @property
    def avg(self):
        return self.score
    @property
    def mc_avg(self):
        return self.mc_score

In [43]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [44]:
class ToxicSimpleNNModel(nn.Module):
    def __init__(self, use_aux=True):
        super(ToxicSimpleNNModel, self).__init__()
        self.backbone = XLMRobertaModel.from_pretrained(BACKBONE_PATH)
        self.dropout = nn.Dropout(0.3)
        aux_len = 0

        if use_aux:
            aux_len = 5
        self.linear = nn.Linear(
            in_features=self.backbone.pooler.dense.out_features*2,
            out_features=2+aux_len,
        )

    def forward(self, input_ids, attention_masks):
        bs, seq_length = input_ids.shape
        seq_x, _ = self.backbone(input_ids=input_ids, attention_mask=attention_masks)
        apool = torch.mean(seq_x, 1)
        mpool, _ = torch.max(seq_x, 1)
        x = torch.cat((apool, mpool), 1)
        x = self.dropout(x)

        return self.linear(x)

In [45]:
import warnings

In [46]:
warnings.filterwarnings("ignore")

In [47]:
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

In [48]:
from catalyst.data.sampler import DistributedSamplerWrapper, BalanceClassSampler

In [49]:
class TPUFitter:

    def __init__(self, model, device, config):
        if not os.path.exists('node_submissions'):
            os.makedirs('node_submissions')

        self.config = config
        self.epoch = 0
        self.log_path = 'log.txt'

        self.model = model
        self.device = device

        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]

        self.optimizer = AdamW(optimizer_grouped_parameters, lr=config.lr*xm.xrt_world_size())
        self.scheduler = config.SchedulerClass(self.optimizer, **config.scheduler_params)

        self.criterion = config.criterion
        xm.master_print(f'Fitter prepared. Device is {self.device}')

    def fit(self, train_loader, validation_loader):
        for e in range(self.config.n_epochs):
            if self.config.verbose:
                lr = self.optimizer.param_groups[0]['lr']
                timestamp = datetime.utcnow().isoformat()
                self.log(f'\n{timestamp}\nLR: {lr}')

            t = time.time()
            para_loader = pl.ParallelLoader(train_loader, [self.device])
            losses, final_scores = self.train_one_epoch(para_loader.per_device_loader(self.device))

            self.log(f'[RESULT]: Train. Epoch: {self.epoch}, loss: {losses.avg:.5f}, final_score: {final_scores.avg:.5f}, mc_score: {final_scores.mc_avg:.5f}, time: {(time.time() - t):.5f}')

            t = time.time()
            para_loader = pl.ParallelLoader(validation_loader, [self.device])
            losses, final_scores = self.validation(para_loader.per_device_loader(self.device))

            self.log(f'[RESULT]: Validation. Epoch: {self.epoch}, loss: {losses.avg:.5f}, final_score: {final_scores.avg:.5f}, mc_score: {final_scores.mc_avg:.5f}, time: {(time.time() - t):.5f}')

            if self.config.validation_scheduler:
                self.scheduler.step(metrics=final_scores.mc_avg)

            self.epoch += 1

    def run_tuning_and_inference(self, test_loader, validation_tune_loader):
        for e in range(2):
            self.optimizer.param_groups[0]['lr'] = self.config.lr*xm.xrt_world_size()
            para_loader = pl.ParallelLoader(validation_tune_loader, [self.device])
            losses, final_scores = self.train_one_epoch(para_loader.per_device_loader(self.device))
            para_loader = pl.ParallelLoader(test_loader, [self.device])
            self.run_inference(para_loader.per_device_loader(self.device))

    def validation(self, val_loader):
        self.model.eval()
        losses = AverageMeter()
        final_scores = RocAucMeter()

        t = time.time()

        for step, (inputs_masks, targets) in enumerate(val_loader):
            inputs=inputs_masks[0]
            attention_masks=inputs_masks[1]

            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    xm.master_print(
                        f'Valid Step {step}, loss: ' + \
                        f'{losses.avg:.5f}, final_score: {final_scores.avg:.5f}, mc_score: {final_scores.mc_avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}'
                    )
            with torch.no_grad():
                inputs = inputs.to(self.device, dtype=torch.long)
                attention_masks = attention_masks.to(self.device, dtype=torch.long)
                targets = targets.to(self.device, dtype=torch.float)

                outputs = self.model(inputs, attention_masks)
                loss = self.criterion(outputs, targets)

                batch_size = inputs.size(0)

                final_scores.update(targets, outputs)
                losses.update(loss.detach().item(), batch_size)

        return losses, final_scores

    def train_one_epoch(self, train_loader):
        self.model.train()

        losses = AverageMeter()
        final_scores = RocAucMeter()
        t = time.time()

        for step, (inputs_masks, targets) in enumerate(train_loader):
            inputs=inputs_masks[0]
            attention_masks=inputs_masks[1]
            batch_size = inputs.size(0)

            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    logger.info(
                        f'Train Step {step}, bs: {batch_size}, loss: ' + \
                        f"{losses.avg:.5f}, lr: {self.optimizer.param_groups[0]['lr']} final_score: {final_scores.avg:.5f}, mc_score: {final_scores.mc_avg:.5f}, " + \
                        f'time: {(time.time() - t):.5f}'
                    )

            inputs = inputs.to(self.device, dtype=torch.long)
            attention_masks = attention_masks.to(self.device, dtype=torch.long)
            targets = targets.to(self.device, dtype=torch.float)

            self.optimizer.zero_grad()

            outputs = self.model(inputs, attention_masks)
            loss = self.criterion(outputs, targets)


            final_scores.update(targets, outputs)

            losses.update(loss.detach().item(), batch_size)

            loss.backward()
            _check_grad(self.optimizer)
            logger.info("step: %d, loss: %f", step, loss)

            xm.optimizer_step(self.optimizer)

            if self.config.step_scheduler:
                self.scheduler.step()

        self.model.eval()
        self.save('last-checkpoint.bin')

        return losses, final_scores

    def run_inference(self, test_loader):
        self.model.eval()
        result = {'id': [], 'toxic': []}
        t = time.time()

        for step, (inputs_masks, ids) in enumerate(test_loader):
            inputs=inputs_masks[0]
            attention_masks=inputs_masks[1]

            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    xm.master_print(f'Prediction Step {step}, time: {(time.time() - t):.5f}')

            with torch.no_grad():
                inputs = inputs.to(self.device, dtype=torch.long)
                attention_masks = attention_masks.to(self.device, dtype=torch.long)
                outputs = self.model(inputs, attention_masks)
                toxics = nn.functional.softmax(outputs, dim=1).data.cpu().numpy()[:,1]

            result['id'].extend(ids.cpu().numpy())
            result['toxic'].extend(toxics)

        result = pd.DataFrame(result)
        node_count = len(glob('node_submissions/*.csv'))
        result.to_csv(f'node_submissions/submission_{node_count}_{datetime.utcnow().microsecond}_{random.random()}.csv', index=False)

    def save(self, path):
        xm.save(self.model.state_dict(), path)

    def log(self, message):
        if self.config.verbose:
            xm.master_print(message)
        with open(self.log_path, 'a+') as logger:
            xm.master_print(f'{message}', logger)

In [50]:
class LabelSmoothing(nn.Module):
    """https://github.com/pytorch/pytorch/issues/7455#issuecomment-513062631"""

    def __init__(self, smoothing = 0.1, dim=-1):
        super(LabelSmoothing, self).__init__()
        self.cls = 2
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.dim = dim

    def forward(self, x, target):
        if self.training:
            pred = x[:,:2].log_softmax(dim=self.dim)
            aux=x[:, 2:]

            toxic_target = target[:,:2]
            aux_target = target[:, 2:]
            with torch.no_grad():
                # smooth_toxic = pred.data.clone()
                smooth_toxic = self.smoothing + (1-self.smoothing*2)*toxic_target
                # smooth_toxic.scatter_(1, toxic_target.data.unsqueeze(1), self.confidence) # only for 0 1 label, put confidence to related place
                # for 0-1, 0 -> 0.1, 1->0.9.(if 1), if zero. 0->0.9, 1->0.1
                smooth_aux = self.smoothing + (1-self.smoothing*2)*aux_target  # only for binary cross entropy, so for lable, it is (1-smooth)*

            aux_loss = torch.nn.functional.binary_cross_entropy_with_logits(aux, smooth_aux)

            return torch.mean(torch.sum(-smooth_toxic * pred, dim=self.dim)) + aux_loss/3
        else:
            return torch.nn.functional.cross_entropy(x[:,:2], target[:,:2])

In [51]:
class TrainGlobalConfig:
    """ Global Config for this notebook """
    num_workers = 0  # количество воркеров для loaders
    batch_size = 16  # bs
    n_epochs = 2  # количество эпох для обучения
    lr = 0.5 * 1e-5 # стартовый learning rate (внутри логика работы с мульти TPU домножает на кол-во процессов)
    fold_number = 0  # номер фолда для обучения

    # -------------------
    verbose = True  # выводить принты
    verbose_step = 1  # количество шагов для вывода принта
    # -------------------

    # --------------------
    step_scheduler = False  # выполнять scheduler.step после вызова optimizer.step
    validation_scheduler = True  # выполнять scheduler.step после валидации loss (например для плато)
    SchedulerClass = torch.optim.lr_scheduler.ReduceLROnPlateau
    scheduler_params = dict(
        mode='max',
        factor=0.7,
        patience=0,
        verbose=False,
        threshold=0.0001,
        threshold_mode='abs',
        cooldown=0,
        min_lr=1e-8,
        eps=1e-08
    )
    # --------------------

    # -------------------
    criterion = LabelSmoothing()
    # -------------------

In [52]:
def test_init():
    l = Shonenkov(loss_func=None, metrics=None)
    assert l is not None

In [53]:
!cp /kaggle/input/clean-pickle-for-jigsaw-toxicity/*pkl .

In [54]:
import ipdb

In [55]:
from kaggle_runner import may_debug

In [56]:
k = Shonenkov(metrics=None, loss_func=LabelSmoothing(), opt_func=None)
k.run(dump_flag=False)

[DEBUG]2020-06-19 09:08:22,678:utils:load ot.pkl
[DEBUG]2020-06-19 09:08:24,180:utils:None -> KernelRunningState.SAVE_SUBMISSION_DONE
[DEBUG]2020-06-19 09:08:24,182:utils:load train.pkl
[DEBUG]2020-06-19 09:08:28,460:utils:load val.pkl
[DEBUG]2020-06-19 09:08:28,804:utils:state KernelRunningState.PREPARE_DATA_DONE
[DEBUG]2020-06-19 09:09:27,520:utils:state KernelRunningState.TRAINING_DONE
[DEBUG]2020-06-19 09:09:27,522:utils:state KernelRunningState.EVL_DEV_DONE
[DEBUG]2020-06-19 09:09:27,523:utils:load test.pkl
[DEBUG]2020-06-19 09:09:28,138:utils:state KernelRunningState.SAVE_SUBMISSION_DONE


In [57]:
def _check_grad(raw_opt):
    pg = raw_opt.param_groups
    pg0pl = pg[0]['params'] # pg0pl[0] is a Parameter
    pg1pl = pg[1]['params'] # pg0pl[0] is a Parameter

    #with torch.no_grad():
    #    #norms = torch.tensor([torch.norm(p) for p in pg0pl])
    #    normsg = torch.tensor([torch.norm(p.grad) for p in pg0pl if p.grad is not None])
    #    #logger.debug("params info pg0: norm std(%f) mean(%f)", *torch.std_mean(norms))
    #    logger.debug("grad info pg0: norm std(%f) mean(%f)", *torch.std_mean(normsg))

    #    #norms1 = torch.tensor([torch.norm(p) for p in pg1pl])
    #    norms1g = torch.tensor([torch.norm(p.grad) for p in pg1pl if p.grad is not None])
    #    #logger.debug("params info pg1: norm std(%f) mean(%f)", *torch.std_mean(norms1))
    #    logger.debug("grad info pg1: norm std(%f) mean(%f)", *torch.std_mean(norms1g))

In [58]:
from kaggle_runner import logger

def test_model_fn(device=torch.device("cpu")):
    #device = xm.xla_device(devkind='TPU')
    #device=torch.device("xla")
    logger.debug("Device used: %s", device)

    #k.run(dump_flag=True) # it seems it cannot save right
    #k.run(dump_flag=False)
    #k.peek_data()

    self = k
    assert self.validation_dataset is not None
    #assert self.learner is not None

    net = k.model
    assert net is not None
    net.to(device)

    param_optimizer = list(self.model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    #optimizer = AdamW(optimizer_grouped_parameters, lr=TrainGlobalConfig.lr*xm.xrt_world_size())
    optimizer = AdamW(optimizer_grouped_parameters, lr=TrainGlobalConfig.lr*8)

    train_loader = torch.utils.data.DataLoader(
        self.train_dataset,
        batch_size=TrainGlobalConfig.batch_size,
        shuffle=False, # sampler is set, so shuffle here should be False
        sampler=BalanceClassSampler(labels=k.train_dataset.get_labels(), mode="downsampling"),
        pin_memory=False,
        drop_last=True,
        num_workers=TrainGlobalConfig.num_workers,
    )
    may_debug()
    validation_loader = torch.utils.data.DataLoader(
        self.validation_dataset,
        batch_size=TrainGlobalConfig.batch_size,
    #    sampler=validation_sampler,
        pin_memory=False,
        drop_last=False,
        num_workers=TrainGlobalConfig.num_workers
    )
    test_loader = torch.utils.data.DataLoader(
        self.test_dataset,
        batch_size=TrainGlobalConfig.batch_size,
    #    sampler=test_sampler,
        pin_memory=False,
        drop_last=False,
        num_workers=TrainGlobalConfig.num_workers
    )
    validation_tune_loader = torch.utils.data.DataLoader(
        self.validation_tune_dataset,
        batch_size=TrainGlobalConfig.batch_size,
        #sampler=validation_tune_sampler,
        pin_memory=False,
        drop_last=False,
        num_workers=TrainGlobalConfig.num_workers
    )

    def validation(model, device, config, val_loader, criterion):
        model.eval()
        losses = AverageMeter()
        final_scores = RocAucMeter()

        t = time.time()

        for step, (inputs_masks, targets) in enumerate(val_loader):
            inputs=inputs_masks[0]
            attention_masks=inputs_masks[1]

            if config.verbose:
                if step % config.verbose_step == 0:
                    logger.info(
                        f'Valid Step {step}, loss: ' + \
                        f'{losses.avg:.5f}, final_score: {final_scores.avg:.5f}, mc_score: {final_scores.mc_avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}'
                    )
            with torch.no_grad():
                inputs = inputs.to(device, dtype=torch.long)
                attention_masks = attention_masks.to(device, dtype=torch.long)
                targets = targets.to(device, dtype=torch.float)

                outputs = model(inputs, attention_masks)
                loss = criterion(outputs, targets)

                batch_size = inputs.size(0)

                final_scores.update(targets, outputs)
                losses.update(loss.detach().item(), batch_size)

    def run_inference(model, device, config, test_loader):
        model.eval()
        result = {'id': [], 'toxic': []}
        t = time.time()

        for step, (inputs_masks, targets) in enumerate(test_loader):
            inputs=inputs_masks[0]
            attention_masks=inputs_masks[1]

            if config.verbose:
                if step % config.verbose_step == 0:
                    logger.info(f'Prediction Step {step}, time: {(time.time() - t):.5f}')

            with torch.no_grad():
                inputs = inputs.to(device, dtype=torch.long)
                attention_masks = attention_masks.to(device, dtype=torch.long)
                outputs = model(inputs, attention_masks)
                toxics = nn.functional.softmax(outputs, dim=1).data.cpu().numpy()[:,1]

            result['id'].extend(ids.cpu().numpy())
            result['toxic'].extend(toxics)

        return result

    def train_one_epoch(model, device, config, train_loader, criterion, optimizer):
        model.train()

        losses = AverageMeter()
        final_scores = RocAucMeter()
        t = time.time()

        for step, (inputs_masks, targets) in enumerate(train_loader):
            inputs=inputs_masks[0]
            attention_masks=inputs_masks[1]

            batch_size = inputs.size(0)

            if config.verbose:
                if step % config.verbose_step == 0:
                    logger.debug(
                        f'Train Step {step}, bs: {batch_size}, loss: ' + \
                        f"{losses.avg:.5f}, lr: {optimizer.param_groups[0]['lr']} final_score: {final_scores.avg:.5f}, mc_score: {final_scores.mc_avg:.5f}, " + \
                        f'time: {(time.time() - t):.5f}'
                    )

            inputs = inputs.to(device, dtype=torch.long)
            attention_masks = attention_masks.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            optimizer.zero_grad()

            outputs = model(inputs, attention_masks)
            loss = criterion(outputs, targets)


            final_scores.update(targets, outputs)

            losses.update(loss.detach().item(), batch_size)

            loss.backward()
            _check_grad(optimizer)
            #optimizer.step()
            xm.optimizer_step(optimizer, barrier=True)

        model.eval()
        #self.save('last-checkpoint.bin')

        return losses, final_scores

    def run_tuning_and_inference(net, device, TrainGlobalConfig, validation_loader, train_loader):
        for e in range(1):
            self.optimizer.param_groups[0]['lr'] = self.config.lr*8

            losses, final_scores = train_one_epoch(net, device, TrainGlobalConfig, train_loader, TrainGlobalConfig.criterion, )
            self.log(f'[RESULT]: Tune_Train. Epoch: {self.epoch}, loss: {losses.avg:.5f}, final_score: {final_scores.avg:.5f}, mc_score: {final_scores.mc_avg:.5f}, time: {(time.time() - t):.5f}')

            t = time.time()
            para_loader = pl.ParallelLoader(validation_loader, [self.device])
            losses, final_scores = self.validation(para_loader.per_device_loader(self.device))
            self.log(f'[RESULT]: Tune_Validation. Epoch: {self.epoch}, loss: {losses.avg:.5f}, final_score: {final_scores.avg:.5f}, mc_score: {final_scores.mc_avg:.5f}, time: {(time.time() - t):.5f}')

            run_inference(net, device, TrainGlobalConfig, validation_loader)

    train_one_epoch(net, device, TrainGlobalConfig, train_loader, TrainGlobalConfig.criterion, optimizer)
    losses, final_scores = validation(net, device, TrainGlobalConfig, validation_loader, TrainGlobalConfig.criterion)
    logger.info(f"Val results: losses={losses}, final_scores={final_scores}")

    results = run_inference(net, device, TrainGlobalConfig, validation_loader)
    logger.info(f"Test done, result len %d", len(results))

In [59]:
from kaggle_runner import defaults
_DEBUG = defaults.DEBUG
defaults.DEBUG = True
#test_model_fn()
defaults.DEBUG = _DEBUG

In [60]:
#k.learner
#k.learner.recorder.plot()

In [61]:
import warnings
warnings.filterwarnings('ignore')

import torch_xla
import torch_xla.distributed.data_parallel as dp
import torch_xla.utils.utils as xu
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch

import fastai
from fastai import *
from fastai.core import *
from fastai.torch_core import *
from fastai.vision import *
from fastai.basic_train import *
from kaggle_runner import logger

def len_parallelloader(self):
    return len(self._loader._loader)
pl.PerDeviceLoader.__len__ = len_parallelloader

In [78]:
import pysnooper
class CheckGrad(LearnerCallback):
    def __init__(self, learn:Learner, skip_loss_step=False):
        super().__init__(learn)
        self.skip_loss_step = skip_loss_step
        logger.debug("Callback CheckGrad skip_loss_step: " +str(self.skip_loss_step))
        self.losses = None
        self.final_scores = None

    def on_train_begin(self, **kwargs:Any)->None:
        self.losses = AverageMeter()
        self.final_scores = RocAucMeter()

    def on_backward_begin(self, **kwargs:Any)->None:
        #print(kwargs.keys())
        """dict_keys(['epoch', 'iteration', 'num_batch', 'skip_validate',
        'n_epochs', 'pbar', 'metrics', 'stop_training', 'last_input',
        'last_target', 'train', 'stop_epoch', 'skip_step', 'skip_zero',
        'skip_bwd', 'last_output', 'last_loss', 'smooth_loss'])
        """
        pg = self.learn.opt.opt.param_groups
        #logger.debug("grad info: %s", raw_opt)
        logger.debug(f"on_backward_begin lr: {pg[0]['lr']}")
        logger.debug("itr: %d, num_batch: %d, last loss: %f, smooth_loss: %f",
                     kwargs['iteration'], kwargs['num_batch'],
                     kwargs['last_loss'], kwargs['smooth_loss'])

        self.final_scores.update(kwargs['last_target'], kwargs['last_output'])
        self.losses.update(kwargs['last_loss'].detach().item(), TrainGlobalConfig.batch_size)
        logger.debug(f"loss_avg: {self.losses.avg:.5f}, lr_pg0:"
                     f"{pg[0]['lr']}, lr_pg1: {pg[1]['lr']}final_score:"
                     f"{self.final_scores.avg:.5f}, mc_score:"
                     f"{self.final_scores.mc_avg:.5f}")

    def on_backward_end(self, **kwargs:Any)->None:
        raw_opt = self.learn.opt.opt
        _check_grad(raw_opt)

        return {'skip_step': self.skip_loss_step}


import pysnooper
class TPUDistributed(LearnerCallback):
    def __init__(self, learn:Learner, debug=True):
        super().__init__(learn)

        self.debug = debug

        if debug:
            self.device = xm.xla_device(devkind='TPU')
            logger.debug("TPUDistributed in DEBUG mode")
            #self.device = xm.xla_device(devkind='CPU')
        else:
            self.device = xm.xla_device(devkind='TPU')
        logger.debug("%s used for xla_device for TPUDistributed" % self.device)

    def _change_dl(self,dl, shuffle):
        old_dl = dl
        train_sampler = DistributedSamplerWrapper(
            sampler=BalanceClassSampler(labels=k.train_dataset.get_labels(), mode="downsampling"),
            num_replicas=xm.xrt_world_size(),
            rank=xm.get_ordinal(),
            shuffle=True
        )
        train_loader = torch.utils.data.DataLoader(
            k.train_dataset,
            batch_size=TrainGlobalConfig.batch_size,
            sampler=train_sampler,
            pin_memory=False,
            drop_last=True,
            num_workers=TrainGlobalConfig.num_workers,
        )
        new_dl = train_loader

        return old_dl,new_dl,train_sampler

    def _change_dl_val(self,dl, shuffle):
        old_dl = dl
        validation_sampler = torch.utils.data.distributed.DistributedSampler(
            k.validation_dataset,
            num_replicas=xm.xrt_world_size(),
            rank=xm.get_ordinal(),
            shuffle=False
        )
        validation_loader = torch.utils.data.DataLoader(
            k.validation_dataset,
            batch_size=TrainGlobalConfig.batch_size,
            sampler=validation_sampler,
            pin_memory=False,
            drop_last=False,
            num_workers=TrainGlobalConfig.num_workers
        )

        return old_dl,validation_loader,validation_sampler

    def on_train_begin(self, **kwargs:Any)->None:
        self.learn.model = self.learn.model.to(self.device)

        pg = self.learn.opt.opt.param_groups
        pg0pl = pg[0]['params'] # pg0pl[0] is a Parameter
        pg1pl = pg[1]['params'] # pg0pl[0] is a Parameter

        #logger.debug("grad info: %s", raw_opt)
        logger.debug(f"on_train_begin pg0 lr: {pg[0]['lr']}")
        logger.debug(f"on_train_begin pg1 lr: {pg[1]['lr']}")

        if self.debug:
            self.learn.opt.lr = self.learn.opt.lr*xm.xrt_world_size()
            #pg[0]['lr'] *= xm.xrt_world_size() # will do it twice...
            #pg[1]['lr'] *= xm.xrt_world_size()
            logger.debug("opt info: %s\n type: %s", self.learn.opt, type(self.learn.opt))
        else:
            self.learn.opt.lr = self.learn.opt.lr*xm.xrt_world_size()
        logger.debug("%s used for xla_device, to device done" % self.device)
        shuffle = self.data.train_dl.init_kwargs['shuffle'] if hasattr(self.data.train_dl, 'init_kwargs') else True
        self.old_sampler_train_dl,self.data.train_dl,self.train_sampler = self._change_dl(self.data.train_dl, shuffle)

        if hasattr(self.data, 'valid_dl') and self.data.valid_dl is not None:
            self.old_sampler_valid_dl,self.data.valid_dl,self.valid_sampler = self._change_dl_val(self.data.valid_dl, shuffle)

    def on_epoch_begin(self,**kwargs:Any)->None:
        logger.debug("Epoch begins on device %s" % self.device)

        self.old_train_dl = self.data.train_dl
        self.learn.data.train_dl = pl.ParallelLoader(self.old_train_dl, [self.device]).per_device_loader(self.device)
        self.learn.data.train_dl.dataset = None #self.old_train_dl.dataset

        if hasattr(self.data, 'valid_dl') and self.data.valid_dl is not None:
            self.old_valid_dl = self.learn.data.valid_dl
            self.learn.data.valid_dl = pl.ParallelLoader(self.old_valid_dl, [self.device]).per_device_loader(self.device)

            self.learn.data.valid_dl.dataset = self.old_valid_dl.dataset
            self.learn.data.valid_dl.dl = self.learn.data.valid_dl._loader._loader

    def on_backward_end(self, **kwargs:Any)->None:
        xm.optimizer_step(self.learn.opt.opt, barrier=True) # copied from https://github.com/tmabraham/fastai_tpu/blob/8b73018cf705da1a73d9be1f105a8e886051a90c/fastai_v1/tpu_distributed_fastai.py, and needed a fix
        #may_debug(True)

        return {'skip_step': True}

    def on_epoch_end(self,**kwargs:Any)->None:
        self.learn.data.train_dl = self.old_train_dl
        self.learn.data.valid_dl = self.old_valid_dl

    def on_train_end(self,**kwargs:Any)->None:
        self.learn.data.train_dl = self.old_sampler_train_dl
        self.learn.data.valid_dl = self.old_sampler_valid_dl


def _to_tpu_distributed(learn:Learner) -> Learner:
  #Learner.fit = _fit_tpu
    learn.callback_fns.append(TPUDistributed)

    return learn


Learner.to_tpu_distributed = _to_tpu_distributed

def setup_food():
    path = untar_data(URLs.FOOD)

def filelist2df(path):
    df = pd.read_csv(path, delimiter='/', header=None, names=['label', 'name'])
    df['name'] =  df['label'].astype(str) + "/" + df['name'].astype(str) + ".jpg"

    return df

#train_path = path/'train.txt'
#test_path = path/'test.txt'

In [79]:
from functools import partial
from fastai.callbacks.misc import StopAfterNBatches
from fastai.callbacks import *

import pysnooper

@pysnooper.snoop()
def debug_train(use_dist_cb=True):
    logger.debug(f'debug train with{" " if use_dist_cb else "OUT"} to_tpu_distributed')
    from kaggle_runner import defaults
    _DEBUG = defaults.DEBUG
    defaults.DEBUG = True

    param_optimizer = list(k.model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'lr': 0., 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'lr': 0., 'weight_decay': 0.0}
    ]

    def AdamW_with_given_p(p_to_ignore, *args, **kargs):
        kargs['lr']=TrainGlobalConfig.lr*xm.xrt_world_size()

        return AdamW(optimizer_grouped_parameters, *args, **kargs)

    learn = k.create_learner(k, opt_func=AdamW_with_given_p,
                             loss_func=LabelSmoothing(),
                             wd=0.01,
                             callback_fns=[partial(GradientClipping, clip=0.5),
                                           ShowGraph,
                                           partial(CSVLogger, append=True),
                                           partial(CheckGrad, skip_loss_step=False)]
                             )

    if use_dist_cb:
        learn = learn.to_tpu_distributed()

    learn.callbacks.append(StopAfterNBatches(n_batches=200))
    #learn.callback_fns.append(CheckGrad)
    #print('hello')
    #learn.lr_find(start_lr=1e-7, end_lr=1e-4, num_it=200)
    #learn.recorder.plot()
    #learn.fit_one_cycle(1, max_lr=2e-5)
    learn.fit(1, lr=4e-5) # original 0.5*e-5*8=4*e-5
    defaults.DEBUG = _DEBUG

In [80]:
%%time
#debug_train(use_dist_cb=True)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.72 µs


In [81]:
from functools import partial
import pysnooper

@pysnooper.snoop()
def train_loop(index, *args):
  #data = (ImageList.from_df(df=train_df, path=path/'images', cols=1)
  #        .random_split_by_pct(0.2)
  #        .label_from_df(cols=0)
  #        .transform(get_transforms(), size=224)
  #        .databunch(bs=32, num_workers=0)
  #        .normalize(imagenet_stat))
  #learn = cnn_learner(data, models.resnet152, metrics=accuracy).to_tpu_distributed()
    logger.debug("rank: %d entered train_loop", index)

    param_optimizer = list(k.model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'lr': 4e-5, 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'lr': 4e-5, 'weight_decay': 0.0}
    ]

    def AdamW_with_given_p(p_to_ignore, *args, **kargs):
        kargs['lr']=TrainGlobalConfig.lr*xm.xrt_world_size()

        return AdamW(optimizer_grouped_parameters, *args, **kargs)

    if index == 0:
        time.sleep(1)
    learn = k.create_learner(k, opt_func=AdamW_with_given_p,
                             loss_func=LabelSmoothing(),
                             wd=0.01,
                             callback_fns=[partial(GradientClipping, clip=0.5),
                                           ShowGraph,
                                           partial(CSVLogger, append=True),
                                           partial(CheckGrad, skip_loss_step=False)]
                             ).to_tpu_distributed()
    learn.lr_find(start_lr=1e-7, end_lr=1e-5, num_it=200)
    learn.recorder.plot()
    #learn.fit_one_cycle(3, max_lr=5e-6, wd=0.001)
    learn.fit(1, lr=5e-6, wd=0.001)

In [ ]:
FLAGS={}
xmp.spawn(train_loop, args=(FLAGS,),  nprocs=8, start_method='fork')

Source path:... <ipython-input-81-c5300591d4d7>
Starting var:.. index = 0
Starting var:.. args = ({},)
09:55:58.478805 call         5 def train_loop(index, *args):
09:55:58.493702 line        13     logger.debug("rank: %d entered train_loop", index)
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/python_logging_rabbitmq/handlers.py", line 166, in emit
    headers=self.message_headers
  File "/usr/local/lib/python3.6/dist-packages/pika/adapters/blocking_connection.py", line 2247, in basic_publish
    mandatory=mandatory)
  File "/usr/local/lib/python3.6/dist-packages/pika/channel.py", line 428, in basic_publish
    mandatory=mandatory), (properties, body))
  File "/usr/local/lib/python3.6/dist-packages/pika/channel.py", line 1401, in _send_method
    self.connection._send_method(self.channel_number, method, content)
  File "/usr/local/lib/python3.6/dist-packages/pika/connection.py", line 2232, in _send_method
    self._send_message

Starting var:.. index = 4
Starting var:.. args = ({},)
09:55:59.545032 call         5 def train_loop(index, *args):
09:55:59.551240 line        13     logger.debug("rank: %d entered train_loop", index)
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/python_logging_rabbitmq/handlers.py", line 166, in emit
    headers=self.message_headers
  File "/usr/local/lib/python3.6/dist-packages/pika/adapters/blocking_connection.py", line 2247, in basic_publish
    mandatory=mandatory)
  File "/usr/local/lib/python3.6/dist-packages/pika/channel.py", line 428, in basic_publish
    mandatory=mandatory), (properties, body))
  File "/usr/local/lib/python3.6/dist-packages/pika/channel.py", line 1401, in _send_method
    self.connection._send_method(self.channel_number, method, content)
  File "/usr/local/lib/python3.6/dist-packages/pika/connection.py", line 2232, in _send_method
    self._send_message(channel_number, method, content)
  File "/usr/l

  File "/usr/local/lib/python3.6/dist-packages/python_logging_rabbitmq/handlers.py", line 166, in emit
    headers=self.message_headers
  File "/usr/local/lib/python3.6/dist-packages/pika/adapters/blocking_connection.py", line 2247, in basic_publish
    mandatory=mandatory)
--- Logging error ---
  File "/usr/local/lib/python3.6/dist-packages/pika/channel.py", line 428, in basic_publish
    mandatory=mandatory), (properties, body))
  File "/usr/local/lib/python3.6/dist-packages/pika/channel.py", line 1401, in _send_method
    self.connection._send_method(self.channel_number, method, content)
  File "/usr/local/lib/python3.6/dist-packages/pika/connection.py", line 2232, in _send_method
    self._send_message(channel_number, method, content)
  File "/usr/local/lib/python3.6/dist-packages/pika/connection.py", line 2265, in _send_message
    self._output_marshaled_frames(marshaled_body_frames)
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/pika/connection.

  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 233, in dispatch_shell
    handler(stream, idents, msg)
  File "/usr/local/lib/python3.6/dist-packages/torch_xla/distributed/xla_multiprocessing.py", line 293, in spawn
    start_method=start_method)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 399, in execute_request
    user_expressions, allow_stdin)
  File "/usr/local/lib/python3.6/dist-packages/torch/multiprocessing/spawn.py", line 149, in start_processes
    process.start()
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/ipkernel.py", line 208, in do_execute
    res = shell.run_cell(code, store_history=store_history, silent=silent)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 105, in start
    self._popen = self._Popen(self)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/zmqshell.py", line 537, in run_cell
    return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
  File "/usr

  File "/usr/local/lib/python3.6/dist-packages/pika/adapters/utils/selector_ioloop_adapter.py", line 329, in set_writer
    fd, self._readable_mask | self._writable_mask)
  File "/usr/local/lib/python3.6/dist-packages/pika/adapters/select_connection.py", line 528, in update_handler
    self._poller.update_handler(fd, events)
  File "/usr/local/lib/python3.6/dist-packages/pika/adapters/select_connection.py", line 721, in update_handler
    events_to_set=events_set)
  File "/usr/local/lib/python3.6/dist-packages/pika/adapters/select_connection.py", line 1238, in _modify_fd_events
    self._poll.modify(fileno, events)
FileNotFoundError: [Errno 2] No such file or directory
Call stack:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  F

...

[DEBUG]2020-06-19 10:14:19,044:utils:loss_avg: 1.16361, lr_pg0:4e-05, lr_pg1: 4e-05final_score:0.47975, mc_score:-0.00955
[DEBUG]2020-06-19 10:14:19,179:utils:on_backward_begin lr: 4e-05
[DEBUG]2020-06-19 10:14:19,183:utils:on_backward_begin lr: 4e-05
[DEBUG]2020-06-19 10:14:19,206:utils:loss_avg: 1.15343, lr_pg0:4e-05, lr_pg1: 4e-05final_score:0.48855, mc_score:-0.00430
[DEBUG]2020-06-19 10:14:19,187:utils:itr: 574, num_batch: 574, last loss: 1.015625, smooth_loss: 1.181797
[DEBUG]2020-06-19 10:14:19,195:utils:itr: 580, num_batch: 580, last loss: 1.109375, smooth_loss: 1.185507
[DEBUG]2020-06-19 10:14:19,435:utils:loss_avg: 1.18026, lr_pg0:4e-05, lr_pg1: 4e-05final_score:0.47438, mc_score:-0.04115
[DEBUG]2020-06-19 10:14:19,442:utils:loss_avg: 1.16229, lr_pg0:4e-05, lr_pg1: 4e-05final_score:0.48752, mc_score:-0.01089
[DEBUG]2020-06-19 10:14:19,581:utils:on_backward_begin lr: 4e-05
[DEBUG]2020-06-19 10:14:19,612:utils:itr: 570, num_batch: 570, last loss: 1.257812, smooth_loss: 1.164789

[DEBUG]2020-06-19 10:14:25,555:utils:itr: 573, num_batch: 573, last loss: 1.312500, smooth_loss: 1.187535
[DEBUG]2020-06-19 10:14:25,665:utils:loss_avg: 1.16219, lr_pg0:4e-05, lr_pg1: 4e-05final_score:0.48729, mc_score:-0.01174
[DEBUG]2020-06-19 10:14:25,675:utils:loss_avg: 1.17647, lr_pg0:4e-05, lr_pg1: 4e-05final_score:0.48246, mc_score:-0.02634
[DEBUG]2020-06-19 10:14:25,835:utils:loss_avg: 1.18513, lr_pg0:4e-05, lr_pg1: 4e-05final_score:0.47283, mc_score:-0.03173
[DEBUG]2020-06-19 10:14:26,035:utils:on_backward_begin lr: 4e-05
[DEBUG]2020-06-19 10:14:26,134:utils:on_backward_begin lr: 4e-05
[DEBUG]2020-06-19 10:14:26,093:utils:itr: 576, num_batch: 576, last loss: 1.125000, smooth_loss: 1.148970
[DEBUG]2020-06-19 10:14:26,159:utils:itr: 587, num_batch: 587, last loss: 1.218750, smooth_loss: 1.164284
[DEBUG]2020-06-19 10:14:26,356:utils:loss_avg: 1.15379, lr_pg0:4e-05, lr_pg1: 4e-05final_score:0.48805, mc_score:-0.00486
[DEBUG]2020-06-19 10:14:26,413:utils:loss_avg: 1.16361, lr_pg0:4

[DEBUG]2020-06-19 10:14:32,063:utils:loss_avg: 1.16358, lr_pg0:4e-05, lr_pg1: 4e-05final_score:0.47428, mc_score:-0.02725
[DEBUG]2020-06-19 10:14:32,059:utils:itr: 579, num_batch: 579, last loss: 0.984375, smooth_loss: 1.174637
[DEBUG]2020-06-19 10:14:32,097:utils:on_backward_begin lr: 4e-05
[DEBUG]2020-06-19 10:14:32,132:utils:on_backward_begin lr: 4e-05
[DEBUG]2020-06-19 10:14:32,120:utils:loss_avg: 1.15349, lr_pg0:4e-05, lr_pg1: 4e-05final_score:0.48840, mc_score:-0.00461
[DEBUG]2020-06-19 10:14:32,139:utils:itr: 585, num_batch: 585, last loss: 1.187500, smooth_loss: 1.178450
[DEBUG]2020-06-19 10:14:32,140:utils:itr: 581, num_batch: 581, last loss: 1.335938, smooth_loss: 1.182904
[DEBUG]2020-06-19 10:14:32,333:utils:loss_avg: 1.16271, lr_pg0:4e-05, lr_pg1: 4e-05final_score:0.48012, mc_score:-0.00958
[DEBUG]2020-06-19 10:14:32,397:utils:loss_avg: 1.18038, lr_pg0:4e-05, lr_pg1: 4e-05final_score:0.47450, mc_score:-0.04140
[DEBUG]2020-06-19 10:14:32,384:utils:loss_avg: 1.17721, lr_pg0:4

[DEBUG]2020-06-19 10:14:38,586:utils:loss_avg: 1.16202, lr_pg0:4e-05, lr_pg1: 4e-05final_score:0.48862, mc_score:-0.01011
[DEBUG]2020-06-19 10:14:39,129:utils:on_backward_begin lr: 4e-05
[DEBUG]2020-06-19 10:14:39,141:utils:itr: 580, num_batch: 580, last loss: 0.847656, smooth_loss: 1.185205
[DEBUG]2020-06-19 10:14:39,412:utils:loss_avg: 1.18495, lr_pg0:4e-05, lr_pg1: 4e-05final_score:0.47269, mc_score:-0.03143
[DEBUG]2020-06-19 10:14:39,441:utils:on_backward_begin lr: 4e-05
[DEBUG]2020-06-19 10:14:39,447:utils:itr: 581, num_batch: 581, last loss: 0.980469, smooth_loss: 1.165101
[DEBUG]2020-06-19 10:14:39,571:utils:on_backward_begin lr: 4e-05
[DEBUG]2020-06-19 10:14:39,579:utils:itr: 583, num_batch: 583, last loss: 1.093750, smooth_loss: 1.175639
[DEBUG]2020-06-19 10:14:39,696:utils:loss_avg: 1.16347, lr_pg0:4e-05, lr_pg1: 4e-05final_score:0.47407, mc_score:-0.02824
[DEBUG]2020-06-19 10:14:39,849:utils:loss_avg: 1.16289, lr_pg0:4e-05, lr_pg1: 4e-05final_score:0.47978, mc_score:-0.00903

[DEBUG]2020-06-19 10:14:44,601:utils:loss_avg: 1.15436, lr_pg0:4e-05, lr_pg1: 4e-05final_score:0.48835, mc_score:-0.00423
[DEBUG]2020-06-19 10:14:45,163:utils:on_backward_begin lr: 4e-05
[DEBUG]2020-06-19 10:14:45,168:utils:itr: 588, num_batch: 588, last loss: 1.132812, smooth_loss: 1.183657


In [75]:
import pysnooper

@pysnooper.snoop()
def _mp_fn(rank, flags, k=k):
    device = xm.xla_device(devkind='TPU')
    logger.debug("%s used for xla_device" % device)
    net = k.model
    net.to(device)
    logger.debug("%s used for xla_device, to device done" % device)

    train_sampler = DistributedSamplerWrapper(
        sampler=BalanceClassSampler(labels=k.train_dataset.get_labels(), mode="downsampling"),
        num_replicas=xm.xrt_world_size(),
        rank=xm.get_ordinal(),
        shuffle=True
    )
    train_loader = torch.utils.data.DataLoader(
        k.train_dataset,
        batch_size=TrainGlobalConfig.batch_size,
        sampler=train_sampler,
        pin_memory=False,
        drop_last=True,
        num_workers=TrainGlobalConfig.num_workers,
    )
    validation_sampler = torch.utils.data.distributed.DistributedSampler(
        k.validation_dataset,
        num_replicas=xm.xrt_world_size(),
        rank=xm.get_ordinal(),
        shuffle=False
    )
    validation_loader = torch.utils.data.DataLoader(
        k.validation_dataset,
        batch_size=TrainGlobalConfig.batch_size,
        sampler=validation_sampler,
        pin_memory=False,
        drop_last=False,
        num_workers=TrainGlobalConfig.num_workers
    )
    validation_tune_sampler = torch.utils.data.distributed.DistributedSampler(
        k.validation_tune_dataset,
        num_replicas=xm.xrt_world_size(),
        rank=xm.get_ordinal(),
        shuffle=True
    )
    validation_tune_loader = torch.utils.data.DataLoader(
        k.validation_tune_dataset,
        batch_size=TrainGlobalConfig.batch_size,
        sampler=validation_tune_sampler,
        pin_memory=False,
        drop_last=False,
        num_workers=TrainGlobalConfig.num_workers
    )
    test_sampler = torch.utils.data.distributed.DistributedSampler(
        k.test_dataset,
        num_replicas=xm.xrt_world_size(),
        rank=xm.get_ordinal(),
        shuffle=False
    )
    test_loader = torch.utils.data.DataLoader(
        k.test_dataset,
        batch_size=TrainGlobalConfig.batch_size,
        sampler=test_sampler,
        pin_memory=False,
        drop_last=False,
        num_workers=TrainGlobalConfig.num_workers
    )

    logger.debug("rank: %d", rank)

    if rank == 0:
        time.sleep(1)

    fitter = TPUFitter(model=net, device=device, config=TrainGlobalConfig)
    fitter.fit(train_loader, validation_loader)
    fitter.run_tuning_and_inference(test_loader, validation_tune_loader)

In [76]:
import gc
gc.collect()

331

In [ ]:
%%time

if __name__ == "__main__":
    FLAGS={}
    xmp.spawn(_mp_fn, args=(FLAGS,),  nprocs=8, start_method='fork')

Source path:... <ipython-input-75-22a604177f6b>
Starting var:.. rank = 0
Starting var:.. flags = {}
Starting var:.. k = <__main__.Shonenkov object at 0x7f0d94184898>
09:33:55.096281 call         4 def _mp_fn(rank, flags, k=k):
09:33:55.104124 line         5     device = xm.xla_device(devkind='TPU')
New var:....... device = device(type='xla', index=1)
09:33:55.105500 line         6     logger.debug("%s used for xla_device" % device)
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/python_logging_rabbitmq/handlers.py", line 166, in emit
    headers=self.message_headers
  File "/usr/local/lib/python3.6/dist-packages/pika/adapters/blocking_connection.py", line 2247, in basic_publish
    mandatory=mandatory)
  File "/usr/local/lib/python3.6/dist-packages/pika/channel.py", line 428, in basic_publish
    mandatory=mandatory), (properties, body))
  File "/usr/local/lib/python3.6/dist-packages/pika/channel.py", line 1401, in _send_method
  

[DEBUG]2020-06-19 09:33:55,107:utils:xla:1 used for xla_device
09:33:55.194616 line         7     net = k.model
New var:....... net = ToxicSimpleNNModel(  (backbone): XLMRobertaModel...ear(in_features=2048, out_features=7, bias=True))
09:33:55.195598 line         8     net.to(device)
Source path:... <ipython-input-75-22a604177f6b>
Starting var:.. rank = 1
Starting var:.. flags = {}
Starting var:.. k = <__main__.Shonenkov object at 0x7f0d94184898>
09:33:58.784761 call         4 def _mp_fn(rank, flags, k=k):
09:33:58.791998 line         5     device = xm.xla_device(devkind='TPU')
New var:....... device = device(type='xla', index=0)
09:33:58.793195 line         6     logger.debug("%s used for xla_device" % device)
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/python_logging_rabbitmq/handlers.py", line 166, in emit
    headers=self.message_headers
  File "/usr/local/lib/python3.6/dist-packages/pika/adapters/blocking_connection.py", 

  File "/usr/lib/python3.6/logging/__init__.py", line 865, in handle
    self.emit(record)
  File "/usr/local/lib/python3.6/dist-packages/python_logging_rabbitmq/handlers.py", line 172, in emit
    self.handleError(record)
Message: 'xla:0 used for xla_device'
Arguments: ()
[DEBUG]2020-06-19 09:33:58,795:utils:xla:0 used for xla_device
09:33:58.874969 line         7     net = k.model
New var:....... net = ToxicSimpleNNModel(  (backbone): XLMRobertaModel...ear(in_features=2048, out_features=7, bias=True))
09:33:58.875964 line         8     net.to(device)
Source path:... <ipython-input-75-22a604177f6b>
Starting var:.. rank = 2
Starting var:.. flags = {}
Starting var:.. k = <__main__.Shonenkov object at 0x7f0d94184898>
09:33:59.325526 call         4 def _mp_fn(rank, flags, k=k):
09:33:59.332922 line         5     device = xm.xla_device(devkind='TPU')
New var:....... device = device(type='xla', index=0)
09:33:59.334225 line         6     logger.debug("%s used for xla_device" % device)
--- L

  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/pika/channel.py", line 428, in basic_publish
    mandatory=mandatory), (properties, body))
  File "/usr/local/lib/python3.6/dist-packages/torch/multiprocessing/spawn.py", line 20, in _wrap
    fn(i, *args)
  File "/usr/local/lib/python3.6/dist-packages/pika/channel.py", line 1401, in _send_method
    self.connection._send_method(self.channel_number, method, content)
  File "/usr/local/lib/python3.6/dist-packages/torch_xla/distributed/xla_multiprocessing.py", line 228, in _start_fn
    fn(gindex, *args)
  File "/usr/local/lib/python3.6/dist-packages/pika/connection.py", line 2232, in _send_method
    self._send_message(channel_number, method, content)
  File "/usr/local/lib/python3.6/dist-packages/pysnooper/tracer.py", line 263, in simple_wrapper
    return function(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-pack

  File "<ipython-input-75-22a604177f6b>", line 6, in _mp_fn
    logger.debug("%s used for xla_device" % device)
  File "/usr/lib/python3.6/logging/__init__.py", line 1296, in debug
    self._log(DEBUG, msg, args, **kwargs)
  File "/usr/lib/python3.6/logging/__init__.py", line 1444, in _log
    self.handle(record)
  File "/usr/lib/python3.6/logging/__init__.py", line 1454, in handle
    self.callHandlers(record)
  File "/usr/lib/python3.6/logging/__init__.py", line 1516, in callHandlers
    hdlr.handle(record)
  File "/usr/lib/python3.6/logging/__init__.py", line 865, in handle
    self.emit(record)
  File "/usr/local/lib/python3.6/dist-packages/python_logging_rabbitmq/handlers.py", line 172, in emit
    self.handleError(record)
Message: 'xla:0 used for xla_device'
Arguments: ()
[DEBUG]2020-06-19 09:33:59,390:utils:xla:0 used for xla_device
09:33:59.474471 line         7     net = k.model
New var:....... net = ToxicSimpleNNModel(  (backbone): XLMRobertaModel...ear(in_features=2048, out_

...

[INFO]2020-06-19 09:44:00,338:utils:Train Step 161, bs: 16, loss: 0.66645, lr: 4e-05 final_score: 0.91466, mc_score: 0.67531, time: 524.72086
[INFO]2020-06-19 09:44:00,338:utils:Train Step 161, bs: 16, loss: 0.66612, lr: 4e-05 final_score: 0.91582, mc_score: 0.67951, time: 518.23826
[INFO]2020-06-19 09:44:00,338:utils:Train Step 161, bs: 16, loss: 0.66417, lr: 4e-05 final_score: 0.91581, mc_score: 0.68029, time: 527.28872
[INFO]2020-06-19 09:44:00,339:utils:Train Step 161, bs: 16, loss: 0.66330, lr: 4e-05 final_score: 0.91447, mc_score: 0.69210, time: 518.49241
[INFO]2020-06-19 09:44:00,339:utils:Train Step 161, bs: 16, loss: 0.68031, lr: 4e-05 final_score: 0.90358, mc_score: 0.65427, time: 520.58154
[INFO]2020-06-19 09:44:00,398:utils:Train Step 161, bs: 16, loss: 0.65903, lr: 4e-05 final_score: 0.92028, mc_score: 0.67246, time: 524.96293
[INFO]2020-06-19 09:44:00,408:utils:Train Step 161, bs: 16, loss: 0.67357, lr: 4e-05 final_score: 0.91034, mc_score: 0.66976, time: 530.13907
[INFO]

[INFO]2020-06-19 09:44:10,779:utils:Train Step 166, bs: 16, loss: 0.67013, lr: 4e-05 final_score: 0.91282, mc_score: 0.67668, time: 540.51009
[INFO]2020-06-19 09:44:10,781:utils:Train Step 166, bs: 16, loss: 0.66383, lr: 4e-05 final_score: 0.91743, mc_score: 0.68541, time: 528.68096
[INFO]2020-06-19 09:44:10,786:utils:Train Step 166, bs: 16, loss: 0.65703, lr: 4e-05 final_score: 0.92182, mc_score: 0.67776, time: 535.35089
[INFO]2020-06-19 09:44:10,788:utils:Train Step 166, bs: 16, loss: 0.67723, lr: 4e-05 final_score: 0.90611, mc_score: 0.66010, time: 531.03037
[INFO]2020-06-19 09:44:10,826:utils:Train Step 166, bs: 16, loss: 0.66014, lr: 4e-05 final_score: 0.91716, mc_score: 0.69620, time: 528.97968
[INFO]2020-06-19 09:44:10,826:utils:Train Step 166, bs: 16, loss: 0.66094, lr: 4e-05 final_score: 0.91804, mc_score: 0.68682, time: 537.77666
[INFO]2020-06-19 09:44:12,021:utils:step: 166, loss: 0.535156
[INFO]2020-06-19 09:44:12,021:utils:step: 166, loss: 0.482422
[INFO]2020-06-19 09:44:1

[INFO]2020-06-19 09:44:21,098:utils:Train Step 171, bs: 16, loss: 0.65756, lr: 4e-05 final_score: 0.92057, mc_score: 0.69312, time: 548.04887
[INFO]2020-06-19 09:44:21,098:utils:Train Step 171, bs: 16, loss: 0.66242, lr: 4e-05 final_score: 0.91845, mc_score: 0.68876, time: 538.99875
[INFO]2020-06-19 09:44:21,115:utils:Train Step 171, bs: 16, loss: 0.65443, lr: 4e-05 final_score: 0.92374, mc_score: 0.68425, time: 545.68012
[INFO]2020-06-19 09:44:21,098:utils:Train Step 171, bs: 16, loss: 0.66863, lr: 4e-05 final_score: 0.91423, mc_score: 0.68181, time: 550.82953
[INFO]2020-06-19 09:44:21,116:utils:Train Step 171, bs: 16, loss: 0.67640, lr: 4e-05 final_score: 0.90783, mc_score: 0.66793, time: 538.97641
[INFO]2020-06-19 09:44:22,324:utils:step: 171, loss: 0.601562
[INFO]2020-06-19 09:44:22,326:utils:step: 171, loss: 0.664062
[INFO]2020-06-19 09:44:22,326:utils:step: 171, loss: 0.550781
[INFO]2020-06-19 09:44:22,325:utils:step: 171, loss: 0.660156
[INFO]2020-06-19 09:44:22,324:utils:step: 

[INFO]2020-06-19 09:44:31,435:utils:Train Step 176, bs: 16, loss: 0.67347, lr: 4e-05 final_score: 0.90963, mc_score: 0.67318, time: 549.29521
[INFO]2020-06-19 09:44:31,450:utils:Train Step 176, bs: 16, loss: 0.66817, lr: 4e-05 final_score: 0.91436, mc_score: 0.68525, time: 561.18108
[INFO]2020-06-19 09:44:31,468:utils:Train Step 176, bs: 16, loss: 0.67262, lr: 4e-05 final_score: 0.90932, mc_score: 0.66825, time: 551.71035
[INFO]2020-06-19 09:44:31,469:utils:Train Step 176, bs: 16, loss: 0.65441, lr: 4e-05 final_score: 0.92114, mc_score: 0.70705, time: 549.62246
[INFO]2020-06-19 09:44:32,671:utils:step: 176, loss: 0.507812
[INFO]2020-06-19 09:44:32,672:utils:step: 176, loss: 0.605469
[INFO]2020-06-19 09:44:32,671:utils:step: 176, loss: 0.847656
[INFO]2020-06-19 09:44:32,671:utils:step: 176, loss: 0.656250
[INFO]2020-06-19 09:44:32,687:utils:step: 176, loss: 0.585938
[INFO]2020-06-19 09:44:32,687:utils:step: 176, loss: 0.671875
[INFO]2020-06-19 09:44:32,701:utils:step: 176, loss: 0.60156

[INFO]2020-06-19 09:44:41,811:utils:Train Step 181, bs: 16, loss: 0.65279, lr: 4e-05 final_score: 0.92432, mc_score: 0.68994, time: 566.37614
[INFO]2020-06-19 09:44:41,821:utils:Train Step 181, bs: 16, loss: 0.66618, lr: 4e-05 final_score: 0.91541, mc_score: 0.68919, time: 571.55203
[INFO]2020-06-19 09:44:41,822:utils:Train Step 181, bs: 16, loss: 0.67009, lr: 4e-05 final_score: 0.91106, mc_score: 0.67330, time: 562.06469
[INFO]2020-06-19 09:44:43,022:utils:step: 181, loss: 0.574219
[INFO]2020-06-19 09:44:43,022:utils:step: 181, loss: 0.605469
[INFO]2020-06-19 09:44:43,022:utils:step: 181, loss: 0.585938
[INFO]2020-06-19 09:44:43,022:utils:step: 181, loss: 0.640625
[INFO]2020-06-19 09:44:43,022:utils:step: 181, loss: 0.835938
[INFO]2020-06-19 09:44:43,024:utils:step: 181, loss: 0.578125
[INFO]2020-06-19 09:44:43,021:utils:step: 181, loss: 0.718750
[INFO]2020-06-19 09:44:43,023:utils:step: 181, loss: 0.664062
[INFO]2020-06-19 09:44:43,843:utils:Train Step 182, bs: 16, loss: 0.65238, lr:

In [ ]:
from datetime import date
today = date.today()
output_model_file='XLMRobertaModel_tpu_trained.bin'
torch.save(k.model.state_dict(), f"{today}_{output_model_file}")

In [ ]:
submission = pd.concat([pd.read_csv(path) for path in glob('node_submissions/*.csv')]).groupby('id').mean()
submission['toxic'].hist(bins=100)

In [ ]:
submission.to_csv(f'{ROOT_PATH}/submission.csv')

#!cp log.txt '/content/drive/My Drive/jigsaw2020-kaggle-public-baseline/'
!make -C kaggle_runner push_dataset